# 🔥 Wildfire Risk Assessment - Test Notebook

This notebook tests the **xclim-integrated fire weather index module** and demonstrates the complete TerraAgent workflow.

**Key Features:**
- Integration with [xclim](https://github.com/Ouranosinc/xclim) fire weather indices
- Canadian Fire Weather Index (FWI) System
- Fire-prone region analysis
- Monte Carlo risk quantification

In [ ]:
# Setup - Add parent directory to path
import sys
sys.path.insert(0, '..')

## 1. Check xclim Installation Status

In [ ]:
from src.science_fire import check_fire_installation, get_fire_locations

# Check installation status
status = check_fire_installation()
print("🔍 xclim Fire Indices Installation Status:")
print(f"  ├─ Package Installed: {'✅' if status['installed'] else '❌'} {status['installed']}")
print(f"  ├─ Data Ready: {'✅' if status['data_ready'] else '❌'} {status['data_ready']}")
print(f"  ├─ Mode: {status['mode']}")
print(f"  └─ Documentation: {status['documentation']}")

if not status['installed']:
    print(f"\n⚠️ To install xclim: {status['install_command']}")

## 2. Available Fire-Prone Locations

In [ ]:
from src.science_fire import FIRE_LOCATIONS

print("📍 Fire-Prone Regions:")
print("="*80)
print(f"{'Location':<22} {'Temp (°C)':>10} {'Humidity':>10} {'Wind':>10} {'Fire Season':>15}")
print("-"*80)

for loc, info in FIRE_LOCATIONS.items():
    print(f"{loc:<22} {info['typical_temp']:>8.0f}°C {info['typical_humidity']:>8}% {info['typical_wind']:>8} km/h   {info['fire_season']:>15}")

## 3. Basic Fire Weather Index Calculation

In [ ]:
from src.science_fire import calculate_fwi_risk

# Calculate FWI for Los Angeles
result = calculate_fwi_risk(
    location_name="Los Angeles, CA",
    temperature=35.0,      # °C
    humidity=25,           # %
    wind_speed=30.0,       # km/h (Santa Ana winds)
    precipitation=0.0,     # mm
    num_simulations=1000
)

print("🔥 Fire Weather Index Result:")
print("="*60)
print(f"  📍 Location: {result['location']}")
print(f"  🌡️ Temperature: {result['temperature']:.1f}°C")
print(f"  💧 Humidity: {result['humidity']}%")
print(f"  💨 Wind Speed: {result['wind_speed']:.1f} km/h")
print(f"  🌧️ Precipitation: {result['precipitation']:.1f} mm")
print(f"  📊 Mean FWI: {result['mean_fwi']:.1f}")
print(f"  📈 95% CI: {result['ci_lower']:.1f} - {result['ci_upper']:.1f}")
print(f"  ⚠️ Risk Level: {result['risk_level']}")
print(f"  🎨 Risk Color: {result['risk_color']}")
print(f"  ⚙️ Using xclim: {'✅' if result.get('using_xclim', False) else '❌'}")

## 4. FWI Risk Level Categories

In [ ]:
print("🔥 Fire Weather Index Risk Categories")
print("="*50)
print("  FWI Range    │ Risk Level    │ Color")
print("-"*50)
print("  0-5          │ Low           │ Green")
print("  5-10         │ Moderate      │ Blue")
print("  10-20        │ High          │ Yellow")
print("  20-30        │ Very High     │ Orange")
print("  30+          │ Extreme       │ Red")
print("="*50)

## 5. Temperature Sensitivity Analysis

In [ ]:
print("🌡️ Temperature Sensitivity (Los Angeles, 25% humidity, 20 km/h wind)")
print("="*70)
print(f"{'Temp (°C)':<12} {'Mean FWI':>12} {'Risk Level':>15} {'95% CI':>20}")
print("-"*70)

for temp in [20, 25, 30, 35, 40, 45]:
    result = calculate_fwi_risk(
        location_name="Los Angeles, CA",
        temperature=float(temp),
        humidity=25,
        wind_speed=20.0,
        precipitation=0.0,
        num_simulations=500
    )
    ci = f"{result['ci_lower']:.1f}-{result['ci_upper']:.1f}"
    print(f"{temp:<12} {result['mean_fwi']:>12.1f} {result['risk_level']:>15}   {ci:>20}")

print("\n💡 Higher temperatures significantly increase fire weather danger.")

## 6. Humidity Sensitivity Analysis

In [ ]:
print("💧 Humidity Sensitivity (Los Angeles, 35°C, 20 km/h wind)")
print("="*70)
print(f"{'Humidity (%)':<15} {'Mean FWI':>12} {'Risk Level':>15} {'95% CI':>20}")
print("-"*70)

for humidity in [10, 20, 30, 40, 50, 60, 70]:
    result = calculate_fwi_risk(
        location_name="Los Angeles, CA",
        temperature=35.0,
        humidity=humidity,
        wind_speed=20.0,
        precipitation=0.0,
        num_simulations=500
    )
    ci = f"{result['ci_lower']:.1f}-{result['ci_upper']:.1f}"
    print(f"{humidity:<15} {result['mean_fwi']:>12.1f} {result['risk_level']:>15}   {ci:>20}")

print("\n💡 Low humidity dramatically increases fire danger.")

## 7. Location Comparison

In [ ]:
print("🌍 Location Comparison (Using typical fire-season conditions)")
print("="*85)
print(f"{'Location':<22} {'Temp':>8} {'Humidity':>10} {'Wind':>8} {'FWI':>8} {'Risk':>15}")
print("-"*85)

for loc, info in FIRE_LOCATIONS.items():
    result = calculate_fwi_risk(
        location_name=loc,
        temperature=info['typical_temp'],
        humidity=info['typical_humidity'],
        wind_speed=info['typical_wind'],
        precipitation=0.0,
        num_simulations=500
    )
    print(f"{loc:<22} {info['typical_temp']:>6.0f}°C {info['typical_humidity']:>8}% {info['typical_wind']:>6.0f} {result['mean_fwi']:>8.1f} {result['risk_level']:>15}")

print("\n💡 Mediterranean climates (LA, Athens) have highest fire risk due to hot, dry conditions.")

## 8. FWI Visualization

In [ ]:
from src.science_fire import create_fwi_gauge

# Calculate FWI under extreme conditions (Santa Ana winds)
result = calculate_fwi_risk(
    location_name="Los Angeles, CA",
    temperature=38.0,
    humidity=15,
    wind_speed=50.0,  # Santa Ana winds
    precipitation=0.0,
    num_simulations=2000
)

# Create gauge visualization
fig = create_fwi_gauge(result)
fig

## 9. Test Agent Builder (UI Generation)

In [ ]:
from src.agent_builder import StreamlitBuilder
import inspect
from src import science_fire

# Create builder (rule-based, no LLM needed)
builder = StreamlitBuilder()

# Get source code of the module
source_code = inspect.getsource(science_fire)

# Analyze the code
analysis = builder.analyze_code(source_code)
print("📊 Code Analysis Result:")
print(f"  • Function Name: {analysis['name']}")
print(f"  • Return Type: {analysis['return_type']}")
print(f"  • Parameters:")
for p in analysis['parameters']:
    print(f"      - {p['name']}: {p['type']} = {p['default']}")

In [ ]:
# Generate Streamlit UI code
ui_code = builder.generate_ui_code(source_code, "Create a fire weather index risk assessment app")

print("🎨 Generated Streamlit Code Preview:")
print("="*60)
print(ui_code[:1500])
print("..." if len(ui_code) > 1500 else "")
print("="*60)
print(f"\n✅ Total length: {len(ui_code)} characters")

In [ ]:
import ast

# Validate the generated code
try:
    tree = ast.parse(ui_code)
    print("✅ Generated code is syntactically valid Python!")
    print(f"   Contains {len(tree.body)} top-level statements")
    
    # Save the generated app
    app_path = "../generated_fire_app.py"
    with open(app_path, 'w', encoding='utf-8') as f:
        f.write(ui_code)
    print(f"\n📁 Saved generated app to: {app_path}")
    
except SyntaxError as e:
    print(f"❌ Syntax Error: Line {e.lineno}: {e.msg}")

## 10. Generate Launcher Scripts

In [ ]:
# Generate launcher scripts for easy app startup
import os

domain = "fire"
app_name = "generated_fire_app.py"

# Windows batch script
bat_content = f'''@echo off
REM TerraAgent - {domain.title()} Analysis Launcher
REM Double-click this file to start the application

echo ============================================
echo   TerraAgent - {domain.title()} Analysis Tool
echo ============================================
echo.

if exist ".venv\\Scripts\\activate.bat" (
    echo Activating virtual environment...
    call .venv\\Scripts\\activate.bat
) else if exist "venv\\Scripts\\activate.bat" (
    echo Activating virtual environment...
    call venv\\Scripts\\activate.bat
)

echo Starting {domain.title()} Analysis Interface...
echo Press Ctrl+C to stop the server.
echo.

streamlit run {app_name}
pause
'''

# Unix shell script
sh_content = f'''#!/bin/bash
# TerraAgent - {domain.title()} Analysis Launcher

echo "============================================"
echo "  TerraAgent - {domain.title()} Analysis Tool"
echo "============================================"

if [ -f ".venv/bin/activate" ]; then
    source .venv/bin/activate
elif [ -f "venv/bin/activate" ]; then
    source venv/bin/activate
fi

echo "Starting {domain.title()} Analysis Interface..."
streamlit run {app_name}
'''

# Save launcher files
bat_path = f"../launch_{domain}.bat"
sh_path = f"../launch_{domain}.sh"

with open(bat_path, 'w', encoding='utf-8') as f:
    f.write(bat_content)
print(f"✅ Created Windows launcher: {bat_path}")

with open(sh_path, 'w', encoding='utf-8') as f:
    f.write(sh_content)
print(f"✅ Created Unix launcher: {sh_path}")

print(f"\n🚀 Double-click launch_{domain}.bat to start the app!")

## ✅ Summary

This notebook demonstrates the complete TerraAgent fire risk analysis workflow:

1. **xclim Integration** - Verified fire weather indices availability
2. **FWI Calculation** - Canadian Fire Weather Index system
3. **Temperature Sensitivity** - Higher temps → higher risk
4. **Humidity Sensitivity** - Lower humidity → higher risk
5. **Location Comparison** - Fire-prone regions worldwide
6. **Visualization** - FWI gauge with risk levels
7. **Agent Builder** - Generates valid Streamlit UI
8. **IntegratorAgent** - Orchestrates full workflow

**FWI Components (Canadian System):**
- **FFMC** - Fine Fuel Moisture Code
- **DMC** - Duff Moisture Code
- **DC** - Drought Code
- **ISI** - Initial Spread Index
- **BUI** - Build Up Index
- **FWI** - Fire Weather Index (combined)

**To run the generated app:**
```bash
streamlit run generated_fire_app.py
```
Or double-click `launch_fire.bat` (Windows) / `launch_fire.sh` (Unix)